In [ ]:
pip install --upgrade elasticsearch pandas

In [ ]:
from elasticsearch import Elasticsearch

username = %env ELASTICSEARCH_USERNAME
password = %env ELASTICSEARCH_PASSWORD
host = %env ELASTICSEARCH_HOST
port = %env ELASTICSEARCH_PORT

es = Elasticsearch(f'https://{username}:{password}@{host}:{port}', ca_certs='./http_ca.crt')

res = es.search(index='users', query={ "match_all": {} }, size=20)

hits = res['hits']['hits']

for doc in hits:
    print(doc['_source']['name'])

In [ ]:
import pandas as pd

df = pd.json_normalize(hits)

df.head()

In [ ]:
res = es.search(index='users', query={ "match": { "name": "Jay Benitez" } })
hits = res['hits']['hits']
hits[0]['_source']

In [ ]:
# Lucene syntax
res = es.search(index='users', q="name:Eric Rush")

hits = res['hits']['hits']
hits[0]['_source']

In [ ]:
q = { 
    "bool": {
        "must": {
            "match": {
                "city": "West"
            }
        },
        "filter": {
            "match": {
                "zip": "99921"
            }
        }
    }
}

res = es.search(index='users', query=q)

hits = res['hits']['hits']
hits

In [ ]:
res = es.search(index='users', query={ "match_all": {} }, scroll='20m', size=500)
sid = res['_scroll_id']
print(f'sid: {sid}')
size = res['hits']['total']['value']
print(f'size: {size}')

In [ ]:
page = 1
while size > 0:
    res = es.scroll(scroll_id=sid, scroll='20m')
    sid = res['_scroll_id']
    hits = res['hits']['hits']
    size = len(hits)
    print(f'PAGE {page}')
    print(f'SID: {sid}')
    print(f'PENDING SIZE: {size}')
    # for doc in hits:
    #     print(doc['_source']['name'])
    page += 1